In [ ]:
# === install if needed (only once in the kernel) ===
%pip install pybaseball pandas --quiet

# === imports and setup ===
import time
import warnings
import pandas as pd
from pybaseball import cache, statcast

warnings.filterwarnings("ignore", category=FutureWarning)
cache.enable()  # enable caching so partial progress persists

# === robust chunked fetch with retries ===
def fetch_statcast_with_retries(start_date, end_date, chunk_days=15, max_retries=4, backoff=2):
    frames = []
    current = pd.to_datetime(start_date)
    final = pd.to_datetime(end_date)
    while current <= final:
        chunk_end = min(current + pd.Timedelta(days=chunk_days - 1), final)
        attempt = 0
        success = False
        while attempt < max_retries and not success:
            try:
                print(f"Fetching {current.date()} → {chunk_end.date()} (attempt {attempt + 1})")
                chunk = statcast(current.strftime("%Y-%m-%d"), chunk_end.strftime("%Y-%m-%d"))
                frames.append(chunk)
                success = True
            except Exception as e:
                attempt += 1
                wait = backoff ** attempt
                print(f"  Warning: chunk {current.date()} to {chunk_end.date()} failed: {e!r}. Retrying in {wait}s...")
                time.sleep(wait)
        if not success:
            print(f"  ERROR: giving up on {current.date()} → {chunk_end.date()} after {max_retries} attempts.")
        current = chunk_end + pd.Timedelta(days=1)
    if frames:
        return pd.concat(frames, ignore_index=True)
    else:
        return pd.DataFrame()

# === parameters (adjust as needed) ===
start_date = "2015-04-08"  # Statcast begins 2015; Safeco/T-Mobile Park opening season
end_date = "2025-07-31"    # change to narrower window for debugging if desired

# === fetch all statcast data in chunks ===
full_df = fetch_statcast_with_retries(start_date, end_date, chunk_days=15)

# === filter to Mariners home runs (T-Mobile/Safeco Field) ===
hr_df = full_df[
    (full_df.get("events") == "home_run") &
    (full_df.get("home_team") == "SEA")
].copy()

# === distance and exit velocity fields ===
# Choose estimated_distance if available, else fallback
if "estimated_distance" in hr_df.columns:
    hr_df["distance"] = hr_df["estimated_distance"]
else:
    hr_df["distance"] = hr_df.get("hit_distance_sc", pd.NA)

# Exit velocity is usually 'launch_speed'
if "launch_speed" in hr_df.columns:
    hr_df["exit_velocity"] = hr_df["launch_speed"]
else:
    hr_df["exit_velocity"] = hr_df.get("hit_speed", pd.NA)

# === ensure batter name exists ===
if "player_name" not in hr_df.columns:
    hr_df["player_name"] = hr_df["batter"].astype(str)

# === top 10 longest home runs ===
top10 = hr_df.nlargest(10, "distance").reset_index(drop=True)

# === clean display table ===
display_df = pd.DataFrame({
    "Rank": range(1, len(top10)+1),
    "Batter": top10["player_name"],
    "Date": pd.to_datetime(top10["game_date"]).dt.strftime("%Y-%m-%d"),
    "Distance (ft)": top10["distance"].round(1),
    "Exit Velocity (mph)": top10["exit_velocity"].round(1)
})

# optional: show additional context if present
if "inning" in top10.columns:
    display_df["Inning"] = top10["inning"]
if "pitch_type" in top10.columns:
    display_df["Pitch Type"] = top10["pitch_type"]

# result
display_df

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime
import matplotlib.pyplot as plt
import io
from PIL import Image
import os

# === prepare dataframe ===
if "Note" in display_df_swapped.columns:
    df = display_df_swapped.drop(columns=["Note"]).copy()
else:
    df = display_df_swapped.copy()

# enforce column order
df = df[["Rank", "Batter", "Date", "Distance (ft)", "Exit Velocity (mph)"]].copy()

# pretty formatting
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%b %d, %Y")
def fmt_dist(x):
    if pd.isna(x): return ""
    x = float(x)
    return f"{int(x)}" if x.is_integer() else f"{x:.1f}"
df["Distance (ft)"] = df["Distance (ft)"].apply(fmt_dist)
df["Exit Velocity (mph)"] = df["Exit Velocity (mph)"].apply(lambda v: f"{float(v):.1f}" if pd.notna(v) else "")

# Build Styler and drop index by resetting it (so no visible index)
df_for_style = df.reset_index(drop=True)

def highlight_rows(row):
    if row["Rank"] == 1:
        return ['background-color: #fff7e6;'] * len(row)
    elif row["Rank"] == 10:
        return ['background-color: #e8f4ff;'] * len(row)
    else:
        return [''] * len(row)

styled = (
    df_for_style.style
      .set_table_styles([
          {"selector": "th", "props": [("background-color", "#f0f0f0"),
                                       ("font-weight", "bold"),
                                       ("padding", "8px"),
                                       ("border", "1px solid #444444"),
                                       ("white-space", "normal"),
                                       ("word-break", "keep-all"),
                                       ("text-align", "center")]},
          {"selector": "td", "props": [("padding", "6px"),
                                       ("border", "1px solid #444444"),
                                       ("text-align", "center"),
                                       ("font-family", "DejaVu Sans, Arial, sans-serif")]}
      ])
      .apply(highlight_rows, axis=1)
      .set_properties(**{"white-space": "normal"})
      .format({
          "Rank": "{:.0f}",
      })
)

# Compose title & subtitle in HTML
title_html = "<div style='font-size:24px; font-weight:700; text-align:center; margin-bottom:2px;'>Top 10 Longest Home Runs at T-Mobile Park / Safeco Field</div>"
subtitle_html = "<div style='font-size:16px; font-style:italic; color:#555555; text-align:center; margin-bottom:8px;'>In the Statcast era: 2015 and onwards</div>"

html = f"<div style='width:100%; padding:10px; font-family: DejaVu Sans, Arial, sans-serif;'>{title_html}{subtitle_html}{styled.to_html()}</div>"

# Save HTML
with open("top10_table.html", "w", encoding="utf-8") as f:
    f.write(html)

# Create an SVG with embedded HTML (vector)
svg_content = f"""<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<svg xmlns="http://www.w3.org/2000/svg" width="1200" height="600">
  <foreignObject width="100%" height="100%">
    <div xmlns="http://www.w3.org/1999/xhtml">
      {html}
    </div>
  </foreignObject>
</svg>
"""
with open("top10_table.svg", "w", encoding="utf-8") as f:
    f.write(svg_content)

# Fallback PNG: render via matplotlib text/table for guaranteed visibility
fig, ax = plt.subplots(figsize=(10, 5))
ax.axis("off")
ax.text(0.5, 0.95, "Top 10 Longest Home Runs at T-Mobile Park / Safeco Field", ha="center", va="center",
        fontsize=20, weight="bold", transform=ax.transAxes)
ax.text(0.5, 0.90, "In the Statcast era: 2015 and onwards", ha="center", va="center",
        fontsize=13, style="italic", color="#555555", transform=ax.transAxes)

# Render the DataFrame as a table in matplotlib beneath
table = ax.table(cellText=df.values.tolist(),
                 colLabels=["Rank", "Batter", "Date", "Distance (ft)", "Exit Velocity (mph)"],
                 cellLoc="center", colLoc="center", loc="center",
                 cellColours=[["#fff7e6" if (i == 0) else ("#e8f4ff" if (i == len(df)-1) else ("#fafafa" if i %2==1 else "white")) for _ in df.columns] for i in range(len(df))],
                 colColours=["#f0f0f0"] * len(df.columns))
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 1.2)
for (r,c), cell in table.get_celld().items():
    cell.set_edgecolor("#444444")
    if r == 0:
        cell.set_text_props(weight="bold")

fallback_path = "top10_fallback.png"
fig.savefig(fallback_path, dpi=300, bbox_inches="tight")

# Display best version (HTML is saved, show PNG)
from IPython.display import Image as IPyImageDisplay
display(IPyImageDisplay(filename=fallback_path))
print("Saved HTML at:", os.path.abspath("top10_table.html"))
print("Saved SVG at:", os.path.abspath("top10_table.svg"))
print("Saved fallback PNG at:", os.path.abspath(fallback_path))

NameError: name 'display_df_swapped' is not defined